# WIAG: Import von tabellarischen Daten
Anwendungsfall 2024-01-24: Spanische Bischöfe

In [ ]:
using CSV, DataFrames, Dates

## Funktionsdefinitionen

Die Datei "Csv2WIAG_lib.jl" muss sich im gleichen Verzeichnis befinden wie dieses Notebook.

In [ ]:
# load this only once
if !isdefined(Main, :csv2wiag_loaded)
    lib_file = joinpath(".", "Csv2WIAG_lib.jl")
    include(lib_file)
    csv2wiag_loaded = true
else
    "Csv2WIAG_lib ist schon ausgewertet."
end

## Parameter
**Dieser Abschnitt muss vor jedem Import eines neuen Themas bearbeitet werden.**

ID des Corpus in der Datenbank (= `corpus.corpus_id`)

In [ ]:
corpus_id = "ibe"
corpus_id_diocese = "dioc"

ID des Users, auf den die Änderungen eingetragen werden (`item.created_by` und `item.changed_by`).

| User | ID  |
| ---  | --- |
| Bärbel Kröger, bkroege  | 22 |
| Christian Popp, cpopp   | 45 |
| Georg Hertkorn, ghertko | 23 |

In [ ]:
user_id = 23;

Statuswert, der beim Import gesetzt wird.

In [ ]:
online_status = "online"; import_status = "importiert"; default_status = import_status

**Falls keine Diözesen einzulesen sind, kann dieser Parameter und damit die nächsten beiden Schritte übersprungen werden**. Für die spanischen Bischöfe (2024) werden auch neue Diözesen eingelesen.

Erste beim Import verwendete ID für die neuen Diözesen in der Tabelle `item` und `diocese`. Der Wert wird am besten mit einem sinnvollen Abstand zum letzten vorhandenen Eintrag in `item` gewählt. So muss der Parameter nicht verändert werden, wenn in der Zeit bis zum Einlesen der Daten in die Datenbank noch Datensätze über die Redaktionsumgebung der Web-Anwendung angelegt werden.

In [ ]:
next_diocese_id = 57501

Format für die öffentliche ID und erste für den Import verwendeter
Wert für den Zähler in der öffentlichen ID (`item_corpus.id_public`) für Diözesen.

``` sql
SELECT id, corpus_id, name, id_public_mask, next_id_public FROM corpus WHERE corpus_id = 'dioc';
```

In [ ]:
diocese_id_public_mask = "WIAG-Inst_DIOCGatz-###-###"; next_diocese_id_public = 150

**Erste beim Import verwendete ID für die neuen Personen in der Tabelle `item` setzen**. Der Wert wird am besten mit einem sinnvollen Abstand zum letzten vorhandenen Eintrag in `item` gewählt. So muss der Parameter nicht verändert werden, wenn in der Zeit bis zum Einlesen der Daten in die Datenbank noch Datensätze über die Redaktionsumgebung der Web-Anwendung angelegt werden.

Wenn auch Diözesen eingelesen werden, ist zu beachten, dass die GWDG-Server die ID in Schritten von zwei erhöhen.

In [ ]:
next_person_id = 57801

Format für die öffentliche ID und erster beim Import verwendeter Wert für den Zähler in der öffentlichen ID (`item_corpus.id_public`) für Personen des neuen Corpus.


In [ ]:
id_public_mask = "WIAG-Pers-EPISCIberia-#####-###"; next_id_public = 101

Weitere Kennzeichen des Corpus

In [ ]:
nt_corpus = (
    name = "Bischöfe Iberische Halbinsel",
    note = "Datenbank zu Bischöfen der iberischen Halbinsel im Mittelalter (bis ins 13. Jahrhundert)",
    id_public_mask = id_public_mask,
    corpus_id = corpus_id,
    comment = "Datenbank zu Bischöfen der iberischen Halbinsel im Mittelalter (bis ins 13. Jahrhundert)",
    page_title = "Bischöfe der iberischen Halbinsel im Mittelalter (bis ins 13. Jahrhundert)",
    online_status = online_status,
    default_status = default_status,
    edit_form = "person",
    next_id_public = next_id_public
)

Pfade, um Daten einzulesen (CSV) und zu schreiben (SQL-Insert/Update-Statements)

In [ ]:
input_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb3\\UbietoArteta\\data"
output_path = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb3\\UbietoArteta\\data_sql"

## Neues Corpus bekannt machen

In [ ]:
filename = joinpath(output_path, "insert_corpus.sql")
table_name = "corpus"

In [ ]:
key_value_str = join([(String(k) * " = " * val_sql(v)) for (k, v) in pairs(nt_corpus)], ", ")

In [ ]:
open(filename, write = true) do io
    println(io, "INSERT INTO $(table_name) SET ", key_value_str, ";")
end

## Literatur
Die Werte für `reference_id` müssen abgestimmt sein mit den Werten für `reference_id` in der Tabelle `item_reference`. In der Zieldatenbank müssen die Werte für `reference_id` noch frei sein.

Wenn die Literaturangaben für die Bände schon direkt in der Zieldatenbank vorhanden sind, entfällt dieser Abschnitt.

In [ ]:
filename = joinpath(input_path, "reference_volume.csv")
df_reference_volume = CSV.read(filename, DataFrame)
nrow(df_reference_volume)

Pflichtspalten: Dieser Eintrag ist in der Regel nicht zu bearbeiten. Die CSV-Datei muss diese Spalten enthalten und sie müssen befüllt sein.

In [ ]:
insert_cols_mandatory = [
    :reference_id,
    :full_citation,
    :gs_citation,
    :title_short
]

Zusätzliche Spalten: Dieser Eintrag kann bearbeitet werden, wenn weitere Spalten eingelesen werden sollen. Diese muss es in der Zieldatenbank und in der Quelldatei geben. In der Quelldatei können 
Einträge für diese Spalten fehlen. Die Liste kann auch leer sein.

In [ ]:
insert_cols_additional = [
    :author_editor,
    :year_publication,
]

In [ ]:
insert_cols = vcat(insert_cols_mandatory, insert_cols_additional)

In [ ]:
# Wenn es Literatur in einer Datei reference_volume.csv gibt, dann ist die nächste Zeile durch die übernächste zu ersetzen.
filename = joinpath(output_path, "insert_reference_volume_dummy.sql")
# filename = joinpath(output_path, "insert_reference_volume.sql")
table_name = "reference_volume"
insert_sql(filename, table_name, select(df_reference_volume, insert_cols))

## Diözesen
Für die spanischen Bischöfe (2024) werden auch neue Diözesen eingelesen. Wenn keine Diözesen eingelesen werden, kann dieser Abschnitt übersprungen werden.

In [ ]:
filename = joinpath(input_path, "diocese.csv")
df_diocese = CSV.read(filename, DataFrame)
nrow(df_diocese)

Setze die ID für die Datenbank

In [ ]:
df_diocese.id = range(next_diocese_id; length = nrow(df_diocese), step = 2);

Befülle `item` mit Einträgen für die Diözesen

In [ ]:
now_string = Dates.format(Dates.now(), "Y-mm-dd HH:MM:SS")

In [ ]:
df_item_diocese = DataFrame(
    id = df_diocese.id,
    edit_status = import_status,
    created_by = user_id,
    changed_by = user_id,
    date_created = now_string,
    date_changed = now_string,
);

In [ ]:
filename = joinpath(output_path, "insert_item_diocese.sql")
table_name = "item"
insert_sql(filename, table_name, df_item_diocese)

Befülle Tabelle `item_corpus` mit Einträgen für die Diözesen  
Verwende die Werte aus der Spalte `id_csv` für Einträge in `id_in_corpus`.

In [ ]:
dioc_id_public = [make_id_public(diocese_id_public_mask, c) 
    for c in range(next_diocese_id_public, length = nrow(df_diocese))];
length(dioc_id_public)

In [ ]:
df_item_corpus_diocese = DataFrame(
    item_id = df_diocese.id,
    corpus_id = corpus_id_diocese,
    id_public = dioc_id_public,
    id_in_corpus = df_diocese.id_csv
);

In [ ]:
filename = joinpath(output_path, "insert_item_corpus_diocese.sql")
table_name = "item_corpus"
insert_sql(filename, table_name, df_item_corpus_diocese)

Schreibe den Zähler für die nächste öffentliche ID für Diözesen in der Tabelle `corpus` fort.

In [ ]:
next_diocese_id_public_post = next_diocese_id_public + nrow(df_item_corpus_diocese)

In [ ]:
open(filename, append = true) do io
    println(io, "UPDATE corpus SET next_id_public = $(next_diocese_id_public_post) WHERE corpus_id = 'dioc';")
end

Befülle Tabelle `diocese` mit Einträgen für die Diözesen.

In [ ]:
names(df_diocese)

Pflichtspalten: Dieser Eintrag ist in der Regel nicht zu bearbeiten. Die CSV-Datei muss diese Spalten enthalten und sie müssen befüllt sein.

In [ ]:
insert_cols_mandatory = [
    :id,
    :name
];

Zusätzliche Spalten: Dieser Eintrag kann bearbeitet werden, wenn weitere Spalten eingelesen werden sollen. Diese muss es in der Zieldatenbank und in der Quelldatei geben. In der Quelldatei können 
Einträge für diese Spalten fehlen. Die Liste kann auch leer sein.

In [ ]:
insert_cols_additional = [
    :note,
    :comment,
    :ecclesiastical_province,
    :diocese_status,
    :note_bishopric_seat,
    :date_of_founding
];

In [ ]:
insert_cols = vcat(insert_cols_mandatory, insert_cols_additional);

In [ ]:
filename = joinpath(output_path, "insert_diocese.sql")
table_name = "diocese"
insert_sql(filename, table_name, select(df_diocese, insert_cols))

Befülle Tabelle `item_reference` mit Einträgen für die Diözesen.

Die Quell-Datei muss über die Spalte `id_csv` mit den Daten in "diocese.csv" verknüpft sein, oder gültige Werte in einer Spalte `item_id` enthalten.

In [ ]:
filename = joinpath(input_path, "item_reference_diocese.csv")
df_item_reference_diocese = CSV.read(filename, DataFrame)
nrow(df_item_reference_diocese)

Weise einen Wert für `item_id` zu.

In [ ]:
n = nrow(df_item_reference_diocese)
if !("item_id" in names(df_item_reference_diocese)) || count(ismissing, df_item_reference_diocese.item_id) == n
    df_item_reference_diocese.item_id .= 0;
end
names(df_item_reference_diocese)

In [ ]:
idx_csv = Dict(r.id_csv => i for (i, r) in enumerate(eachrow(df_diocese)))
for (i, row) in enumerate(eachrow(df_item_reference_diocese))
    if ismissing(row[:item_id]) || row[:item_id] == 0
        row_number = get(idx_csv, row.id_csv, 0)
        if row_number == 0
            @warn "Für $(row.name) fehlen 'item_id' und 'id_csv' in Zeile $(i)"
        else
            row[:item_id] = df_diocese[row_number, :id]
        end
    end
end
nrow(df_item_reference_diocese)

Pflichtspalten: Dieser Eintrag ist in der Regel nicht zu bearbeiten. Die CSV-Datei muss diese Spalten enthalten und sie müssen befüllt sein.

In [ ]:
insert_cols_mandatory = [
    :item_id,
    :reference_id
];

Zusätzliche Spalten: Dieser Eintrag kann bearbeitet werden, wenn weitere Spalten eingelesen werden sollen. Diese muss es in der Zieldatenbank und in der Quelldatei geben. In der Quelldatei können Einträge in diesen Spalten fehlen. Die Liste kann auch leer sein.

In [ ]:
insert_cols_additional = [
    :note,
    :page,
    :id_in_reference
];

In [ ]:
insert_cols = vcat(insert_cols_mandatory, insert_cols_additional);

In [ ]:
filename = joinpath(output_path, "insert_item_reference_diocese.sql")
table_name = "item_reference"
insert_sql(filename, table_name, select(df_item_reference_diocese, insert_cols))

## Personen

In [ ]:
filename = joinpath(input_path, "person.csv")
df_person = CSV.read(filename, DataFrame)
nrow(df_person)

In [ ]:
df_person.id = range(next_person_id; length = nrow(df_person), step = 2);

Lies auch die Amtsdaten ein. Die Zeiträume in dieser Datei werden gebraucht, um die Zeitspanne festzulegen, in der eine Person gefunden wird.

In [ ]:
filename = joinpath(input_path, "person_role.csv")
df_person_role = CSV.read(filename, DataFrame)
nrow(df_person_role)

Ordne die IDs der Personen zu.

In [ ]:
join_cols = [
    :id_csv => :id_csv,
    :id => :person_id
];

In [ ]:
df_person_role = leftjoin(df_person_role, select(df_person, join_cols), on = :id_csv);
nrow(df_person_role)

Bestimme einen Zahlenwert für Geburtsdatum und Sterbedatum einer Person

In [ ]:
parse_lower(s) = parsemaybe(s, :lower)
parse_upper(s) = parsemaybe(s, :upper)

In [ ]:
transform!(df_person, :date_birth => ByRow(parse_lower) => :num_date_birth);

In [ ]:
transform!(df_person, :date_death => ByRow(parse_upper) => :num_date_death);

Bestimme einen Zahlenwert für Start- und Enddatum der Amtszeiten und den Sortierschlüssel

In [ ]:
transform!(df_person_role, :date_begin => ByRow(parse_lower) => :num_date_begin);

In [ ]:
transform!(df_person_role, :date_end => ByRow(parse_upper) => :num_date_end);

In [ ]:
date_sort_key_ab(a, b) = date_sort_key(!ismissing(a) ? a : b)
transform!(df_person_role, [:date_begin, :date_end] => ByRow(date_sort_key_ab) => :date_sort_key);

Bestimme die Zeitspanne, in der eine Person gefunden werden soll: Befülle `date_min, date_max` in df_person.

In [ ]:
function find_role(df_pr, person_id)
    return @view df_pr[findall(isequal(person_id), df_pr.person_id), :] 
end
# find earliest and latest date in a person's office list
df_person.date_min = Vector{Union{Missing, Int}}(missing, nrow(df_person))
df_person.date_max = Vector{Union{Missing, Int}}(missing, nrow(df_person))
for p in eachrow(df_person)
    df_pr = find_role(df_person_role, p.id)
    date_min = 2000;
    date_max = 0;
    for pr in eachrow(df_pr)
        if !ismissing(pr.num_date_begin) && pr.num_date_begin < date_min; 
            date_min = pr.num_date_begin 
        end
        if !ismissing(pr.num_date_end) && pr.num_date_end > date_max; 
            date_max = pr.num_date_end 
        end
    end
    
    # date_min can't be smaller than date_birth; date_max can't be larger than date_death
    if date_min == 2000 || (!ismissing(p.num_date_birth) && date_min < p.num_date_birth )
        date_min = p.num_date_birth
    end
    if date_max == 0 || (!ismissing(p.num_date_death) && date_max > p.num_date_death )
        date_max = p.num_date_death
    end
    # find a substitute if a value is missing
    date_min = ismissing(date_min) ? date_max : date_min
    date_max = ismissing(date_max) ? date_min : date_max
    
    p.date_min = date_min
    p.date_max = date_max    
end 

Ausgabe von Dateien zur Kontrolle

In [ ]:
filename = joinpath(input_path, "df_person_2024-01-29.csv")
CSV.write(filename, df_person)
filename = joinpath(input_path, "df_person_role_2024-01-29.csv")
CSV.write(filename, df_person_role)

Befülle `item` mit Einträgen für die Personen

In [ ]:
now_string = Dates.format(Dates.now(), "Y-mm-dd HH:MM:SS")

In [ ]:
df_item = DataFrame(
    id = df_person.id,
    edit_status = import_status,
    created_by = user_id,
    changed_by = user_id,
    date_created = now_string,
    date_changed = now_string,
);

In [ ]:
filename = joinpath(output_path, "insert_item.sql")
table_name = "item"
insert_sql(filename, table_name, df_item)

### Corpus
Befülle Tabelle `item_corpus` mit Einträgen für die Diözesen.
Verwende die Werte aus der Spalte `id_csv` für Einträge im Feld `id_in_corpus`.

In [ ]:
id_public = [make_id_public(id_public_mask, c) 
    for c in range(next_id_public, length = nrow(df_person))];
length(id_public)

In [ ]:
df_item_corpus = DataFrame(
    item_id = df_person.id,
    corpus_id = corpus_id,
    id_public = id_public,
    id_in_corpus = df_person.id_csv
);

In [ ]:
filename = joinpath(output_path, "insert_item_corpus.sql")
table_name = "item_corpus"
insert_sql(filename, table_name, df_item_corpus)

Schreibe den Zähler für die nächste öffentliche ID für Personen in der Tabelle `corpus` fort.

In [ ]:
next_id_public_post = next_id_public + nrow(df_item_corpus)

In [ ]:
open(filename, append = true) do io
    println(io, "UPDATE corpus SET next_id_public = $(next_id_public_post) WHERE corpus_id = 'ibe';")
end

### Person
Befülle Tabelle `person`

Pflichtspalten: Dieser Eintrag ist in der Regel nicht zu bearbeiten. Die CSV-Datei muss diese Spalten enthalten und sie müssen befüllt sein.

In [ ]:
insert_cols_mandatory = [
    :id
];

Zusätzliche Spalten: Dieser Eintrag kann bearbeitet werden, wenn weitere Spalten eingelesen werden sollen. Diese muss es in der Zieldatenbank und in der Quelldatei geben. In der Quelldatei können Einträge in diesen Spalten fehlen. Die Liste kann auch leer sein.

In [ ]:
insert_cols_additional = [
    :givenname,
    :prefixname,
    :familyname,
    :date_death,
    :date_birth,
    :note_dates,
    :num_date_birth,
    :num_date_death,
    :date_min,
    :date_max,
    :academic_title,
    :note_person,
    :comment,
    :note_name,
    :religious_order_id
];

In [ ]:
insert_cols = vcat(insert_cols_mandatory, insert_cols_additional);

In [ ]:
filename = joinpath(output_path, "insert_person.sql")
table_name = "person"
insert_sql(filename, table_name, select(df_person, insert_cols))

### Literaturverweise
Befülle Tabelle `item_reference`

Die Quell-Datei muss über die Spalte "id_csv" mit den Daten in "person.csv" verknüpft sein.

In [ ]:
filename = joinpath(input_path, "item_reference.csv")
df_item_reference = CSV.read(filename, DataFrame)
nrow(df_item_reference)

Weise einen Wert für `item_id` zu.

In [ ]:
n = nrow(df_item_reference)
if !("item_id" in names(df_item_reference)) || count(ismissing, df_item_reference.item_id) == n
    df_item_reference.item_id .= 0;
end
names(df_item_reference)

In [ ]:
idx_csv = Dict(r.id_csv => i for (i, r) in enumerate(eachrow(df_person)))
for (i, row) in enumerate(eachrow(df_item_reference))
    if ismissing(row[:item_id]) || row[:item_id] == 0
        row_number = get(idx_csv, row.id_csv, 0)
        if row_number == 0
            @warn "Für $(row.name) fehlen 'item_id' und 'id_csv' in Zeile $(i)"
        else
            row[:item_id] = df_person[row_number, :id]
        end
    end
end
nrow(df_item_reference)

Pflichtspalten: Dieser Eintrag ist in der Regel nicht zu bearbeiten. Die CSV-Datei muss diese Spalten enthalten und sie müssen befüllt sein.

In [ ]:
insert_cols_mandatory = [
    :item_id,
    :reference_id,
];

Zusätzliche Spalten: Dieser Eintrag kann bearbeitet werden, wenn weitere Spalten eingelesen werden sollen. Diese muss es in der Zieldatenbank und in der Quelldatei geben. In der Quelldatei können Einträge in diesen Spalten fehlen. Die Liste kann auch leer sein.

In [ ]:
insert_cols_additional = [
    :note,
    :page,
    :id_in_reference
];

In [ ]:
insert_cols = vcat(insert_cols_mandatory, insert_cols_additional);

In [ ]:
filename = joinpath(output_path, "insert_item_reference.sql")
table_name = "item_reference"
insert_sql(filename, table_name, select(df_item_reference, insert_cols))

### Namensvarianten
Befülle Tabelle `name_lookup`.
Das Notebook ist zu erweitern, wenn die Datei "person.csv" Namensvarianten enthält.

In [ ]:
df_name_lookup = DataFrame(person_id = Vector{Int}(), name_variant = Vector{String}());

In [ ]:
rgx = r"[\w-]+( [\w-]+)*"
function find_word(s)
    w = ""
    m = match(rgx, s)
    if !isnothing(m)
        w = m.match
    end
    return w
end
for row in eachrow(df_person)
    p_list = String[]
    for part in row[[:givenname, :prefixname, :familyname]]    
        if !ismissing(part) && strip(part) != ""
            push!(p_list, find_word(part))
        end
    end
    variant = join(p_list, " ")
    push!(df_name_lookup, (row[:id], variant))
end
nrow(df_name_lookup)

In [ ]:
df_name_lookup[102:107, :]

Pflichtspalten: Dieser Eintrag ist in der Regel nicht zu bearbeiten. Die CSV-Datei muss diese Spalten enthalten und sie müssen befüllt sein.

In [ ]:
insert_cols = [
    :person_id
    :name_variant
];

In [ ]:
filename = joinpath(output_path, "insert_name_lookup.sql")
table_name = "name_lookup"
insert_sql(filename, table_name, select(df_name_lookup, insert_cols))

### Externe Identifier

In [ ]:
filename = joinpath(input_path, "url_external.csv")
df_url_external = CSV.read(filename, DataFrame)
nrow(df_url_external)

In [ ]:
idx_csv = Dict(r.id_csv => i for (i, r) in enumerate(eachrow(df_person)))
df_url_external.item_id .= 0;
for (i, row) in enumerate(eachrow(df_url_external))
    if ismissing(row[:item_id]) || row[:item_id] == 0
        row_number = get(idx_csv, row.id_csv, 0)
        if row_number == 0
            @warn "Für $(row.name) fehlen 'item_id' und 'id_csv' in Zeile $(i)"
        else
            row[:item_id] = df_person[row_number, :id]
        end
    end
end
nrow(df_url_external)

Pflichtspalten: Dieser Eintrag ist in der Regel nicht zu bearbeiten. Die CSV-Datei muss diese Spalten enthalten und sie müssen befüllt sein.

In [ ]:
insert_cols_mandatory = [
    :item_id,
    :authority_id,
    :value
];

Zusätzliche Spalten: Dieser Eintrag kann bearbeitet werden, wenn weitere Spalten eingelesen werden sollen. Diese muss es in der Zieldatenbank und in der Quelldatei geben. In der Quelldatei können Einträge in diesen Spalten fehlen. Die Liste kann auch leer sein.

In [ ]:
insert_cols_additional = [
    
];

In [ ]:
insert_cols = vcat(insert_cols_mandatory, insert_cols_additional);

In [ ]:
filename = joinpath(output_path, "insert_url_external.sql")
table_name = "url_external"
insert_sql(filename, table_name, select(df_url_external, insert_cols))

## Amtsdaten
Amtsdaten sind oben schon eingelesen worden.

**Diözesen über den Namen zuordnen**  
Dieser Abschnitt ist nur relevant, wenn beim Import neue Diözesen aufgenommen werden.

In [ ]:
select!(df_person_role, Not(:diocese_id));

In [ ]:
join_cols = [
    :name => :diocese_name,
    :id => :diocese_id
];

In [ ]:
df_person_role = leftjoin(df_person_role, select(df_diocese, join_cols), on = :diocese_name)
nrow(df_person_role)

**Daten schreiben**

Pflichtspalten: Dieser Eintrag ist in der Regel nicht zu bearbeiten. Die CSV-Datei muss diese Spalten enthalten und sie müssen befüllt sein.

In [ ]:
insert_cols_mandatory = [
    :person_id,
    :role_id
];

Zusätzliche Spalten: Dieser Eintrag kann bearbeitet werden, wenn weitere Spalten eingelesen werden sollen. Diese muss es in der Zieldatenbank und in der Quelldatei geben. In der Quelldatei können Daten für diese Spalten fehlen. Die Liste kann auch leer sein.

Wenn es grundsätzlich Daten zu dem Amtszeiten (`date_begin`, `date_end`) gibt, sollten auch `num_date_begin` und `num_date_end` sowie `date_sort_key` befüllt sein.

In [ ]:
insert_cols_additional = [
    :diocese_name,
    :diocese_id,
    :date_begin,
    :date_end,
    :num_date_begin,
    :num_date_end,
    :date_sort_key,
    :note    
];

In [ ]:
insert_cols = vcat(insert_cols_mandatory, insert_cols_additional);

In [ ]:
filename = joinpath(output_path, "insert_person_role.sql")
table_name = "person_role"
insert_sql(filename, table_name, select(df_person_role, insert_cols))

### Online?
Setze alle Elemente des neuen Corpus online.

Mit den SQL-Skripten bis hierher werden die Daten des Corpus in die Datenbank aufgenommen, sind aber in der Abfrage erst sichtbar, wenn der Status über die Redaktionmaske auf der Wert von `online_status` gesetzt wird.

Das Skript "set_online.sql", das hier erzeugt wird, macht alle Elemente des Corpus in der Abfrage sichtbar

In [ ]:
filename = joinpath(output_path, "set_$(corpus_id)_online.sql")
now_string = Dates.format(Dates.now(), "Y-mm-dd HH:MM:SS")

In [ ]:
update_online_item_sql = """
UPDATE item SET edit_status = '$(online_status)', is_online = 1, 
date_changed = '$(now_string)', changed_by = $(user_id)
WHERE id IN (SELECT item_id FROM item_corpus WHERE corpus_id = '$(corpus_id)');
"""
insert_item_name_role_sql = """
INSERT INTO item_name_role (item_id_name, item_id_role)
SELECT item_id, item_id FROM item_corpus WHERE corpus_id = '$(corpus_id)';
"""

In [ ]:
open(filename, write = true) do io 
    print(io, update_online_item_sql, insert_item_name_role_sql)
end
filename

### Nicht online?
Setze alle Elemente des neuen Corpus offline.

Das Skript "set_not_online.sql", das hier erzeugt wird, nimmt den online-Status zurück

In [ ]:
filename = joinpath(output_path, "set_$(corpus_id)_not_online.sql")
now_string = Dates.format(Dates.now(), "Y-mm-dd HH:MM:SS")

In [ ]:
update_not_online_item_sql = """
UPDATE item SET edit_status = '$(default_status)', is_online = 0, 
date_changed = '$(now_string)', changed_by = $(user_id)
WHERE id IN (SELECT item_id FROM item_corpus WHERE corpus_id = '$(corpus_id)');
"""
delete_item_name_role_sql = """
DELETE FROM item_name_role WHERE item_id_role IN
(SELECT item_id FROM item_corpus WHERE corpus_id = '$(corpus_id)');
"""

In [ ]:
open(filename, write = true) do io 
    print(io, update_not_online_item_sql, delete_item_name_role_sql)
end
filename